In [2]:
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix, f1_score, precision_score, recall_score
#from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn import tree

import time
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression



#import shap


In [3]:

!pip install bertopic
from bertopic.representation import ZeroShotClassification
from bertopic import BERTopic
from transformers import pipeline
## This takes about one minute



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.1 MB/s eta 0:00:00


In [5]:
df1 = pd.read_csv('shohei.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df1['statistics.views_date_last_refreshed'].value_counts()

statistics.views_date_last_refreshed
2024-10-21    25649
2024-10-24     1350
2024-10-23      891
2024-10-22      494
2024-10-05      229
2024-10-14      214
2024-10-19      203
2024-10-20      201
2024-10-13      160
2024-10-15      157
2024-10-12      152
2024-10-17      145
2024-10-16      127
2024-10-18      120
2024-10-03      113
2024-10-02      113
2024-10-10      109
2024-10-11      106
2024-10-08       89
2024-10-04       87
2024-10-06       75
2024-10-09        1
Name: count, dtype: int64

In [6]:
selected_cols = ['content_type', 'creation_time', 'hashtags', 'is_branded_content',
                 'lang', 'match_type', 'post_owner.type', 'post_owner.username',
                 'statistics.comment_count', 'statistics.like_count', 'statistics.views', 'text']

df_2 = df1[selected_cols].copy()


In [ ]:
df_2

,content_type,creation_time,hashtags,is_branded_content,lang,match_type,post_owner.type,post_owner.username,statistics.comment_count,statistics.like_count,statistics.views,text
0,photos,2024-10-24T13:12:33-07:00,"[""shoheiohtani"",""lawsuit"",""ohtani"",""recordbrea...",False,en,"[""post_text""]",business,NaN,0.0,4.0,NaN,Shohei Ohtani’s historic 50-50 baseball sold f...
1,photos,2024-10-24T13:03:01-07:00,"[""emisi\u00f3nestelarnt5"",""ntelemicro5"",""shohe...",False,es,"[""post_text""]",business,NaN,1.0,11.0,NaN,"Con grandes nombres como Aaron Judge, Juan Sot..."
2,videos,2024-10-24T12:51:00-07:00,"[""misiondeportiva"",""beisbol"",""shoheiohtani"",""d...",False,es,"[""post_text""]",creator,NaN,0.0,2.0,NaN,"""OHTANI ES EL MEJOR JUGADOR DEL JUEGO"" \n\n- A..."
3,photos,2024-10-24T12:43:14-07:00,NaN,False,es,"[""post_text""]",business,NaN,0.0,8.0,NaN,Con el japonés Shohei Ohtani y con Aaron Judge...
4,albums,2024-10-24T12:41:20-07:00,"[""shoheiohtani"",""dodgers"",""mlb"",""baseball""]",False,en,"[""post_text""]",business,NaN,7.0,426.0,NaN,Tyler Glasnow’s thoughts on if there’s any cha...
...,...,...,...,...,...,...,...,...,...,...,...,...
30971,photos,2023-10-23T23:38:17-07:00,"[""shoheiohtani"",""baseballcards"",""rookiecard"",""...",False,en,"[""post_text""]",business,NaN,0.0,45.0,3896.0,Shohei Ohtani 2017 Bowman Chrome Mega Box Refr...
30972,videos,2023-10-23T21:03:15-07:00,"[""sasqatchfabrix"",""needles"",""nepenthes""]",False,en,"[""post_text""]",business,NaN,11.0,NaN,221327.0,〈NEEDLES〉x〈Sasquatchfabrix.〉COLLABORATION PROD...
30973,albums,2023-10-23T20:37:28-07:00,"[""shoheiohtani"",""japan"",""17"",""baseball"",""custo...",False,en,"[""post_text""]",business,NaN,0.0,3.0,372.0,"Los Angeles Angels, Shohei Ohtani 17\n\n#17 #j..."
30974,albums,2023-10-23T20:30:19-07:00,"[""shoheiohtani"",""\u5927\u8c37\u7fd4\u5e73""]",False,ja,"[""post_text""]",personal,NaN,51.0,8835.0,351426.0,🩵‎🤍爽やかー♡\n(📸2021©AQUARIUS)髪の毛長めな感じ💙\nPSの盛り上がりの...


In [ ]:
labels = [ "negative restaurant", "positive restaurant",]

def review_pipe(text):
    output = pipe(text,
                  candidate_labels = labels,
                 )
    labs = output["labels"]
    return labs[0]

In [ ]:
%%time
df_s = df.sample(n=100)
df_s["y_pred"] = df_s.raw_text.apply(review_pipe)

In [ ]:
lab2binary = dict(zip(labels,range(2)))
df_s["y_pred_binary"] = df_s.y_pred.apply(lambda x: lab2binary[x])

In [ ]:

print("F1 Score is: ",  f1_score(df_s.y_pred_binary, df_s.metadata_label) )
print("Precision is: ", precision_score(df_s.y_pred_binary, df_s.metadata_label) )
print("Recall is: ",    recall_score(df_s.y_pred_binary, df_s.metadata_label)  )